In [ ]:
from tabulate import tabulate

EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def get_matrix(input_file_name):
    matrix = []
    with open(input_file_name, "r") as file:
        for line in file:
            matrix.append([c for c in line.strip().replace("\n", "")])
    return matrix


In [ ]:
matrix = get_matrix(EXAMPLE)
print(tabulate(matrix, tablefmt="grid"))

In [ ]:
def find_start(matrix):
    for i, row in enumerate(matrix):
        for j, col in enumerate(row):
            if col == ">":
                return (i, j), "right"
            elif col == "<":
                return (i, j), "left"
            elif col == "^":
                return (i, j), "up"
            elif col == "v":
                return (i, j), "down"
    return (-1, -1), "right"

In [ ]:
start, direction = find_start(matrix)
print(start, direction)

In [ ]:
def find_obstacles(matrix):
    obstacles = set()
    for i, row in enumerate(matrix):
        for j, col in enumerate(row):
            if col == "#":
                obstacles.add((i, j))
    return obstacles

In [ ]:
obstacles = find_obstacles(matrix)
print(obstacles)

In [ ]:
def build_obstacle_map(matrix):
    obstacles = find_obstacles(matrix)
    height = len(matrix)
    width = len(matrix[0])
    obstacle_map = {"rows": [[] for _ in range(height)], "columns": [[] for _ in range(width)]}
    for obstacle in obstacles:
        row = obstacle[0]
        col = obstacle[1]
        i = 0
        while i < len(obstacle_map["rows"][row]) and col > obstacle_map["rows"][row][i]:
            i+=1
        obstacle_map["rows"][row].insert(i, col)
        i = 0
        while i < len(obstacle_map["columns"][col]) and row > obstacle_map["columns"][col][i]:
            i+=1
        obstacle_map["columns"][col].insert(i, row)
    return obstacle_map

In [ ]:
obstacle_map = build_obstacle_map(matrix)
print(obstacle_map)


In [ ]:
def find_next_position(obstacle_map, position, direction):
    current_row = position[0]
    current_col = position[1]
    match direction:
        case "right":
            # Check if an obstacle is in the same row, to the right
            for col in obstacle_map["rows"][current_row]:
                if col > current_col:
                    # If so, return the position right before the obstacle
                    return (current_row, col - 1), "down"
            # If no obstacle, then the guard leaves the grid
            return (-1, -1), "right"
        case "left":
            # Check if an obstacle is in the same row, to the left
            for col in reversed(obstacle_map["rows"][current_row]):
                if col < current_col:
                    # If so, return the position right before the obstacle
                    return (current_row, col + 1), "up"
            # If no obstacle, then the guard leaves the grid
            return (-1, -1), "left"
        case "down":
            # Check if an obstacle is in the same column, down from the current position
            for row in obstacle_map["columns"][current_col]:
                if row > current_row:
                    # If so, return the position right before the obstacle
                    return (row - 1, current_col), "left"
            # If no obstacle, then the guard leaves the grid
            return (-1, -1), "down"
        case "up": 
            # Check if an obstacle is in the same column, up from the current position
            for row in reversed(obstacle_map["columns"][current_col]):
                if row < current_row:
                    # If so, return the position right before the obstacle
                    return (row + 1, current_col), "right"
            # If no obstacle, then the guard leaves the grid
            return (-1, -1), "up"
    return (-1, -1), "up"


In [ ]:
def find_path(obstacle_map, start_position, start_direction):
    path = [(start_position, start_direction)]
    position = start_position
    direction = start_direction
    while position != (-1, -1):
        position, direction = find_next_position(obstacle_map, position, direction)    
        path.append((position, direction))
    return path

In [ ]:
path = find_path(obstacle_map, start, direction)
print(path)

In [ ]:
def find_nb_of_positions(path, height, width):
    visited_positions = set()
    i = 0
    while i < len(path) - 1:
        (current_row, current_col), current_direction = path[i]
        (next_row, next_col), _ = path[i+1]
        if next_row == -1 and next_col == -1:
            match current_direction:
                case "right":
                    for col in range(current_col, width):
                        visited_positions.add((current_row, col))
                case "left":
                    for col in range(0, current_col+1):
                        visited_positions.add((current_row, col))
                case "down":
                    for row in range(current_row, height):
                        visited_positions.add((row, current_col))
                case "up":
                    for row in range(0, current_row+1):
                        visited_positions.add((row, current_col))
        if current_row == next_row:
            for col in range(min(current_col, next_col), max(current_col, next_col)+1):
                visited_positions.add((current_row, col))
        elif current_col == next_col:
            for row in range(min(current_row, next_row), max(current_row, next_row)+1):
                visited_positions.add((row, current_col))
        i+=1
    return len(visited_positions)


In [ ]:
print(find_nb_of_positions(path, len(matrix), len(matrix[0])))

In [ ]:
def part_1(input_file_name):
    matrix = get_matrix(input_file_name)
    height = len(matrix)
    width = len(matrix[0])
    start_position, start_direction = find_start(matrix)
    obstacle_map = build_obstacle_map(matrix)
    path = find_path(obstacle_map, start_position, start_direction)
    print(find_nb_of_positions(path, height, width))

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)

In [ ]:
def find_loop(obstacle_map, start_position, start_direction):
    path = [(start_position, start_direction)]
    position = start_position
    direction = start_direction
    while position != (-1, -1):
        position, direction = find_next_position(obstacle_map, position, direction)    
        if (position, direction) in path:
            return True
        path.append((position, direction))
    return False

In [ ]:
def add_obstacle(obstacle_map, row, col):
    obstacle_map["rows"][row].append(col)
    obstacle_map["columns"][col].append(row)
    obstacle_map["rows"][row].sort()
    obstacle_map["columns"][col].sort()
    return obstacle_map

In [ ]:
def remove_obstacle(obstacle_map, row, col):
    obstacle_map["rows"][row].remove(col)
    obstacle_map["columns"][col].remove(row)
    return obstacle_map

In [ ]:
def part_2(input_file_name):
    matrix = get_matrix(input_file_name)
    height = len(matrix)
    width = len(matrix[0])
    start_position, start_direction = find_start(matrix)
    obstacle_map = build_obstacle_map(matrix)
    result = 0
    for i in range(height):
        for j in range(width):
            if matrix[i][j] == ".":
                obstacle_map = add_obstacle(obstacle_map, i, j)
                if find_loop(obstacle_map, start_position, start_direction):
                    result += 1
                obstacle_map = remove_obstacle(obstacle_map, i, j)
    print(result)

In [ ]:
part_2(EXAMPLE)

In [ ]:
part_2(INPUT)